In [1]:
%load_ext autoreload
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
%autoreload 2
# 10/12/2023 - NEW (2 days of optimization) COMPUTATION WITH MASK ON GPU
%timeit %run ../src/boson_cloud_gpu/main.py

58.9 ms ± 50.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
import cupy

_preprocessing_module = r"""
extern "C"{
    #define G 6.6743e-11
    #define c 299792458.0
    #define hbar 1.0545718176461565e-34
    #define m_sun 1.988409870698051e+30
    #define om0 7.27220521664304e-05
    #define r0 5.5e6
    #define onev 1.60217653e-19
    #define f_int 1e30
    #define duty 0.7
    #define t_obs 365 * 86400 * duty
    #define NAN 0.0/0.0

    __global__ void alpha(
        const float* bh_mass,
        const float* boson_mass,
        const int nrows,
        const int ncols,
        float* _alpha
    ){  
        int x_abs = threadIdx.x + blockDim.x * blockIdx.x;
        int y_abs = threadIdx.y + blockDim.y * blockIdx.y;
        
        if ((x_abs < ncols) && (y_abs < nrows)){
            _alpha[x_abs + ncols * y_abs] = bh_mass[x_abs] + boson_mass[y_abs];
        }
    }
}
"""

preprocessing_module = cupy.RawModule(code=_preprocessing_module)
alpha_kernel = preprocessing_module.get_function("alpha")


def dispatch_kernel(kernel, n_rows: int, n_cols: int, *args):
    block_size = (
        32,
        8,
    )

    grid_x = n_cols // block_size[0]
    if (n_cols % block_size[0]) != 0:
        grid_x += 1
    grid_y = n_rows // block_size[1]
    if (n_rows % block_size[1]) != 0:
        grid_y += 1

    grid_size = (
        grid_x,
        grid_y,
    )
    print(grid_size)

    out_var = cupy.zeros((n_rows, n_cols), dtype=cupy.float32)
    kernel(grid_size, block_size, args + (n_rows, n_cols, out_var))

    return out_var

n_rows = 1_000
n_cols = 21_000
boson_mass = cupy.ones(n_rows, dtype=cupy.float32)
bh_mass = cupy.ones(n_cols, dtype=cupy.float32)
dispatch_kernel(alpha_kernel, n_rows, n_cols, bh_mass, boson_mass)

(657, 125)


array([[2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       ...,
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.]], dtype=float32)